In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, text

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [7]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
inspector = inspect(engine)

In [10]:
# Using the inspector to print the column names within the 'Measurement' table and its types
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [11]:
# Using the inspector to print the column names within the 'Station' table and its types
columns = inspector.get_columns('Station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [22]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by((Measurement.date).desc()).first()
most_recent_date

('2017-08-23',)

In [23]:
# Calculate the last 12 months
last_12_months = dt.date(2017, 8 ,23) - dt.timedelta(days=365)
last_12_months

datetime.date(2016, 8, 23)

In [33]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
# Perform a query to retrieve the data and precipitation scores
# Sort the dataframe by date
precip_data_12_months = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date <= '2017-08-23').\
    filter(Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()

precip_df = pd.DataFrame(precip_data_12_months)
precip_df.set_index('date', inplace=True)
precip_df

# Save the query results as a Pandas DataFrame. Explicitly set the column names
precip_df = pd.DataFrame(precip_data_12_months)

# Use Pandas Plotting with Matplotlib to plot the data
precip_plot = precip_df.plot(x='date', y='prcp', rot=90)
precip_plot.set_xlabel('Date')
precip_plot.set_ylabel('Inches')

plt.show()

,prcp
date,
2016-08-23,0.00
2016-08-23,0.15
2016-08-23,0.05
2016-08-23,NaN
2016-08-23,0.02
...,...
2017-08-22,0.50
2017-08-23,0.00
2017-08-23,0.00


In [45]:
# Use Pandas to calculate the summary statistics for the precipitation data
precip_count = precip_df['prcp'].count()
precip_mean = precip_df['prcp'].mean()
precip_std = precip_df['prcp'].std()
precip_min = precip_df['prcp'].min()
precip_25 = precip_df['prcp'].quantile(0.25)
precip_50 = precip_df['prcp'].quantile(0.50)
precip_75 = precip_df['prcp'].quantile(0.75)
precip_max = precip_df['prcp'].max()

precip_stats = pd.DataFrame({
    "precipitation": [precip_count, 
                      precip_mean, 
                      precip_std, 
                      precip_min, 
                      precip_25, 
                      precip_50, 
                      precip_75, 
                      precip_max]},
    index=["count", "mean", "std", "min", "25%", "50%", "75%", "max"])

precip_stats

,precipitation
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


# Exploratory Station Analysis

In [88]:
stations = session.query(Station.station).count()
stations

9

In [113]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_counts = session.query((Measurement.station), func.count(Measurement.station)).\
                        group_by(Measurement.station).\
                        order_by(func.count(Measurement.station).desc()).all()
station_counts

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [126]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()


[(54.0, 85.0, 71.66378066378067)]

In [83]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_data_12_months = session.query(Measurement.tobs).\
    filter(Measurement.date <= '2017-08-23').\
    filter(Measurement.date >= '2016-08-23').\
    filter(Measurement.station == 'USC00519281').\
    order_by(Measurement.date).all()

temp_data_12mo_df = pd.DataFrame(temp_data_12_months, columns=['temp'])
temp_data_12mo_df

# plt.hist(temp_data_12mo_df, label='tobs')
# plt.xlabel('Temperature')
# plt.ylabel('Frequency')
# plt.legend()
# plt.show()


,temp
0,77.0
1,77.0
2,80.0
3,80.0
4,75.0
...,...
347,77.0
348,77.0
349,76.0
350,76.0


# Close Session

In [16]:
# Close Session
session.close()